Problem 3.1 a


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


file_path = '/content/drive/My Drive/eBayAuctions.csv'
data = pd.read_csv(file_path)

In [ ]:
data = pd.get_dummies(data, columns=['Category', 'currency', 'endDay', 'Duration'])

In [ ]:
print(data.columns)

Index(['sellerRating', 'ClosePrice', 'OpenPrice', 'Competitive?',
       'Category_Antique/Art/Craft', 'Category_Automotive', 'Category_Books',
       'Category_Business/Industrial', 'Category_Clothing/Accessories',
       'Category_Coins/Stamps', 'Category_Collectibles', 'Category_Computer',
       'Category_Electronics', 'Category_EverythingElse',
       'Category_Health/Beauty', 'Category_Home/Garden', 'Category_Jewelry',
       'Category_Music/Movie/Game', 'Category_Photography',
       'Category_Pottery/Glass', 'Category_SportingGoods',
       'Category_Toys/Hobbies', 'currency_EUR', 'currency_GBP', 'currency_US',
       'endDay_Fri', 'endDay_Mon', 'endDay_Sat', 'endDay_Sun', 'endDay_Thu',
       'endDay_Tue', 'endDay_Wed', 'Duration_1', 'Duration_3', 'Duration_5',
       'Duration_7', 'Duration_10'],
      dtype='object')


In [ ]:
pivot_category = data.pivot_table(index='Category_Antique/Art/Craft', values='Competitive?', aggfunc='mean')
pivot_currency = data.pivot_table(index='currency_US', values='Competitive?', aggfunc='mean')
pivot_endDay = data.pivot_table(index=['endDay_Mon', 'endDay_Tue', 'endDay_Wed', 'endDay_Thu', 'endDay_Fri', 'endDay_Sat', 'endDay_Sun'], values='Competitive?', aggfunc='mean')
pivot_duration = data.pivot_table(index=['Duration_1', 'Duration_3', 'Duration_5', 'Duration_7', 'Duration_10'], values='Competitive?', aggfunc='mean')


In [ ]:

print("Pivot table for Category:")
print(pivot_category)
print("\nPivot table for Currency:")
print(pivot_currency)
print("\nPivot table for EndDay:")
print(pivot_endDay)
print("\nPivot table for Duration:")
print(pivot_duration)


Pivot table for Category:
                            Competitive?
Category_Antique/Art/Craft              
0                               0.538162
1                               0.564972

Pivot table for Currency:
             Competitive?
currency_US              
0                0.580882
1                0.519350

Pivot table for EndDay:
                                                                              Competitive?
endDay_Mon endDay_Tue endDay_Wed endDay_Thu endDay_Fri endDay_Sat endDay_Sun              
0          0          0          0          0          0          1               0.485207
                                                       1          0               0.427350
                                            1          0          0               0.466899
                                 1          0          0          0               0.603960
                      1          0          0          0          0               0.480000
           1     

b

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X = data.drop(columns=['Competitive?'])
y = data['Competitive?']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)


In [ ]:
logistic_model = LogisticRegression(max_iter=1000)

logistic_model.fit(X_train, y_train)

y_pred = logistic_model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

print("\nClassification Report:")
print(classification_report(y_val, y_pred))

Accuracy: 0.7490494296577946

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.82      0.75       359
           1       0.82      0.69      0.75       430

    accuracy                           0.75       789
   macro avg       0.75      0.75      0.75       789
weighted avg       0.76      0.75      0.75       789



c

In [ ]:
X_no_price = X.drop(columns=['ClosePrice'])

X_train_no_price, X_val_no_price, y_train_no_price, y_val_no_price = train_test_split(X_no_price, y, test_size=0.4, random_state=42)

logistic_model_no_price = LogisticRegression(max_iter=1000)
logistic_model_no_price.fit(X_train_no_price, y_train_no_price)

y_pred_no_price = logistic_model_no_price.predict(X_val_no_price)

# accuracy
accuracy_no_price = accuracy_score(y_val_no_price, y_pred_no_price)
print("Accuracy without closing price:", accuracy_no_price)

# Classification report
print("\nClassification Report without closing price:")
print(classification_report(y_val_no_price, y_pred_no_price))


Accuracy without closing price: 0.6311787072243346

Classification Report without closing price:
              precision    recall  f1-score   support

           0       0.60      0.58      0.59       359
           1       0.66      0.68      0.67       430

    accuracy                           0.63       789
   macro avg       0.63      0.63      0.63       789
weighted avg       0.63      0.63      0.63       789



d

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

closing_price_coef = logistic_model.coef_[0][np.where(X_train.columns == 'ClosePrice')]

if np.abs(closing_price_coef) > 0.1:
    print("The coefficient for closing price is significant practically.")
else:
    print("The coefficient for closing price is not significant practically.")

if np.abs(closing_price_coef) >= 1.645:
    print("The coefficient for closing price is statistically significant at the 10% significance level.")
else:
    print("The coefficient for closing price is not statistically significant at the 10% significance level.")




The coefficient for closing price is not significant practically.
The coefficient for closing price is not statistically significant at the 10% significance level.


e

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

logistic_model_regularized = LogisticRegressionCV(Cs=10, cv=5, penalty='l1', solver='liblinear', max_iter=1000)
logistic_model_regularized.fit(X_train, y_train)

y_pred_regularized = logistic_model_regularized.predict(X_val)

accuracy_regularized = accuracy_score(y_val, y_pred_regularized)
print("Accuracy of regularized logistic regression:", accuracy_regularized)

print("\nClassification Report of regularized logistic regression:")
print(classification_report(y_val, y_pred_regularized))

selected_predictors = X_train.columns[logistic_model_regularized.coef_[0] != 0]
print("\nSelected predictors by regularized logistic regression:", selected_predictors)


Accuracy of regularized logistic regression: 0.7946768060836502

Classification Report of regularized logistic regression:
              precision    recall  f1-score   support

           0       0.71      0.92      0.80       359
           1       0.92      0.69      0.78       430

    accuracy                           0.79       789
   macro avg       0.81      0.81      0.79       789
weighted avg       0.82      0.79      0.79       789


Selected predictors by regularized logistic regression: Index(['sellerRating', 'ClosePrice', 'OpenPrice', 'currency_EUR', 'endDay_Mon',
       'Duration_5'],
      dtype='object')


f

In [ ]:
auction_settings_coefficients = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': logistic_model_regularized.coef_[0]})
auction_settings_coefficients = auction_settings_coefficients[auction_settings_coefficients['Feature'].str.startswith(('Duration', 'OpenPrice', 'endDay', 'currency'))]

auction_settings_coefficients['AbsoluteCoefficient'] = np.abs(auction_settings_coefficients['Coefficient'])
auction_settings_coefficients = auction_settings_coefficients.sort_values(by='AbsoluteCoefficient', ascending=False)

print("Most influential auction settings based on coefficient magnitude:")
print(auction_settings_coefficients)

Most influential auction settings based on coefficient magnitude:
         Feature  Coefficient  AbsoluteCoefficient
21  currency_EUR    -0.260693             0.260693
2      OpenPrice    -0.118414             0.118414
25    endDay_Mon     0.053697             0.053697
33    Duration_5     0.022175             0.022175
22  currency_GBP     0.000000             0.000000
23   currency_US     0.000000             0.000000
24    endDay_Fri     0.000000             0.000000
26    endDay_Sat     0.000000             0.000000
27    endDay_Sun     0.000000             0.000000
28    endDay_Thu     0.000000             0.000000
29    endDay_Tue     0.000000             0.000000
30    endDay_Wed     0.000000             0.000000
31    Duration_1     0.000000             0.000000
32    Duration_3     0.000000             0.000000
34    Duration_7     0.000000             0.000000
35   Duration_10     0.000000             0.000000
